In [ ]:
import os
import shutil
import pandas as pd
from PIL import Image
import numpy as np
#import matplotlib.pyplot as plt
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt

img_rows = 150
img_cols = 224
color_type = 3
batch_size=48
epochs=300
subject='Plant-pathology-2020'
main_path=os.path.join("E:\\kaggle_imgs",subject)
img_path=os.path.join(main_path,"images")
data_path=os.path.join(main_path,"Data")
saved_path=os.path.join(main_path,"saved_models")
paths=[main_path, img_path,saved_path,data_path]
for fp in paths:
    print(fp)
    if not os.path.exists(fp):        
        os.mkdir(fp)
file_path=os.path.join(saved_path,subject+"eda_200607_")
file_best=os.path.join(saved_path,subject+"eda_200607_")

train_img_pkl=os.path.join(data_path,"train_imgs.npy")
test_img_pkl=os.path.join(data_path,"test_imgs.npy")
train_info_pkl=os.path.join(data_path,"df_train_pickle.csv")

num_classes=4

In [ ]:
import os
import gc
import re

In [ ]:
import cv2
import math
import numpy as np
import scipy as sp
import pandas as pd

In [ ]:
import seaborn as sns
from tqdm import tqdm
import matplotlib.cm as cm
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
tqdm.pandas()
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

np.random.seed(0)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
EPOCHS = 20
SAMPLE_LEN = 100
IMAGE_PATH =img_path
TEST_PATH = data_path+"/test.csv"
TRAIN_PATH = data_path+"/train.csv"
SUB_PATH = data_path+"/sample_submission.csv"

sub = pd.read_csv(SUB_PATH)
test_data = pd.read_csv(TEST_PATH)
train_data = pd.read_csv(TRAIN_PATH)

## Load sample image

In [ ]:
def load_image(image_id):
    file_path=image_id+".jpg"
    image=cv2.imread(IMAGE_PATH+"/"+file_path)
    return cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
train_images=train_data["image_id"][:SAMPLE_LEN].progress_apply(load_image)

In [ ]:
fig=px.imshow(cv2.resize(train_images[0],(205,136)))
fig.show()

## distribution of channel values

In [ ]:
red_values=[np.mean(train_images[idx][:,:,0]) for idx in range(len(train_images))]
green_values=[np.mean(train_images[idx][:,:,1]) for idx in range(len(train_images))]
blue_values=[np.mean(train_images[idx][:,:,2]) for idx in range(len(train_images))]
values=[np.mean(train_images[idx]) for idx in range(len(train_images))]

In [ ]:
fig = ff.create_distplot([values], group_labels=["Channels"], colors=["purple"])
fig.update_layout(showlegend=False, template="simple_white")
fig.update_layout(title_text="Distribution of channel values")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig

In [ ]:
values[0:10]

In [ ]:
df_train = pd.read_csv(TRAIN_PATH)
df_train["CAT"]=df_train[['healthy', 'multiple_diseases', 'rust', 'scab']].values.argmax(axis=1)
all_image=[]
for i, image_name  in enumerate(df_train.image_id):
    if i==100:
        break
    all_image.append(load_image(image_name))
all_image=np.stack(all_image,axis=0)

In [ ]:
all_image.shape

In [ ]:
new_values=all_image.mean(axis=(1,2,3)).astype(np.uint8)
new_color_val=all_image.mean(axis=(1,2)).astype(np.uint8)

In [ ]:
sns.distplot(new_values)

In [ ]:
f,ax=plt.subplots(figsize=(15,5))
sns.distplot(new_color_val[:,0],bins=50,kde=False)

In [ ]:
f,ax=plt.subplots(figsize=(15,5))
sns.distplot(new_color_val[:,1],bins=50,kde=False)

In [ ]:
f,ax=plt.subplots(figsize=(15,5))
sns.distplot(new_color_val[:,2],bins=50,kde=False)

In [ ]:
df_box1=pd.DataFrame({"val":new_color_val[:,0]})
df_box2=pd.DataFrame({"val":new_color_val[:,1]})
df_box3=pd.DataFrame({"val":new_color_val[:,2]})
df_box1["CAT"]="red"
df_box2["CAT"]="green"
df_box3["CAT"]="blue"

In [ ]:
df_box=pd.concat([df_box1,df_box2]).reset_index()
df_box=pd.concat([df_box,df_box3]).reset_index()

In [ ]:
sns.boxplot(x="CAT",y="val",data=df_box)

In [ ]:
fig = ff.create_distplot([red_values, green_values, blue_values],
                         group_labels=["R", "G", "B"],
                         colors=["red", "green", "blue"])
fig.update_layout(title_text="Distribution of red channel values", template="simple_white")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig.data[1].marker.line.color = 'rgb(0, 0, 0)'
fig.data[1].marker.line.width = 0.5
fig.data[2].marker.line.color = 'rgb(0, 0, 0)'
fig.data[2].marker.line.width = 0.5
fig

## visualize sample leaves

In [ ]:
def visualize_leaves(imgs,title=None):
    if title:
        print(title)
    cnt=imgs.shape[0]-1
    row=int(cnt/3) +1
    col=3
    f,ax=plt.subplots(row,col,figsize=(20,5*row))
    for idx in range(cnt+1):
        r=int(idx/3)
        c=idx%3
        ax[r][c].imshow(imgs[idx])
        

In [ ]:
df_healthy=df_train[df_train.healthy==1]
imgs_healthy=[]
for i,(name)  in enumerate(df_healthy.image_id):
    if i==9:
        break
    imgs_healthy.append(load_image(name))
imgs_healthy=np.stack(imgs_healthy,axis=0)   
visualize_leaves(imgs_healthy,"Healthy leaves")

In [ ]:
df_healthy=df_train[df_train.CAT==3]
imgs_healthy=[]
for i,(name)  in enumerate(df_healthy.image_id):
    if i==9:
        break
    imgs_healthy.append(load_image(name))
imgs_healthy=np.stack(imgs_healthy,axis=0)   
visualize_leaves(imgs_healthy,"scab leaves")

In [ ]:
df_healthy=df_train[df_train.CAT==2]
imgs_healthy=[]
for i,(name)  in enumerate(df_healthy.image_id):
    if i==9:
        break
    imgs_healthy.append(load_image(name))
imgs_healthy=np.stack(imgs_healthy,axis=0)   
visualize_leaves(imgs_healthy,"rust leaves")

In [ ]:
df_healthy=df_train[df_train.CAT==1]
imgs_healthy=[]
for i,(name)  in enumerate(df_healthy.image_id):
    if i==9:
        break
    imgs_healthy.append(load_image(name))
imgs_healthy=np.stack(imgs_healthy,axis=0)   
visualize_leaves(imgs_healthy,"multiple diseases leaves")

In [ ]:
df_train["SUM"]=df_train[['healthy', 'multiple_diseases', 'rust', 'scab']].sum(axis=1)

In [ ]:
category=['healthy', 'multiple_diseases', 'rust', 'scab']
df_train.CAT=df_train.CAT.apply(lambda x : category[x])

## count plt for category

In [ ]:
sns.countplot(df_train.CAT)

In [ ]:
df_train["IsHealthy"]=df_train.CAT=="healthy"

In [ ]:
sns.countplot(df_train.IsHealthy)


## Image augmentation

In [ ]:
def edge_and_cut(img):
    emb_img = img.copy()
    edges = cv2.Canny(img, 100, 200)
    edge_coors = []
    for i in range(edges.shape[0]):
        for j in range(edges.shape[1]):
            if edges[i][j] != 0:
                edge_coors.append((i, j))
    
    row_min = edge_coors[np.argsort([coor[0] for coor in edge_coors])[0]][0]
    row_max = edge_coors[np.argsort([coor[0] for coor in edge_coors])[-1]][0]
    col_min = edge_coors[np.argsort([coor[1] for coor in edge_coors])[0]][1]
    col_max = edge_coors[np.argsort([coor[1] for coor in edge_coors])[-1]][1]
    new_img = img[row_min:row_max, col_min:col_max]
    
    emb_img[row_min-10:row_min+10, col_min:col_max] = [255, 0, 0]
    emb_img[row_max-10:row_max+10, col_min:col_max] = [255, 0, 0]
    emb_img[row_min:row_max, col_min-10:col_min+10] = [255, 0, 0]
    emb_img[row_min:row_max, col_max-10:col_max+10] = [255, 0, 0]
    
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(30, 20))
    ax[0].imshow(img, cmap='gray')
    ax[0].set_title('Original Image', fontsize=24)
    ax[1].imshow(edges, cmap='gray')
    ax[1].set_title('Canny Edges', fontsize=24)
    ax[2].imshow(emb_img, cmap='gray')
    ax[2].set_title('Bounding Box', fontsize=24)
    plt.show()


In [ ]:
edge_and_cut(train_images[3])
edge_and_cut(train_images[4])
edge_and_cut(train_images[5])

In [ ]:
train_images[3].shape

## Augmentation  - Flip

In [ ]:
def invert(img):
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(30, 20))
    ax[0].imshow(img)
    ax[0].set_title('Original Image', fontsize=24)
    ax[1].imshow(cv2.flip(img, 0))
    ax[1].set_title('Vertical Flip', fontsize=24)
    ax[2].imshow(cv2.flip(img, 1))
    ax[2].set_title('Horizontal Flip', fontsize=24)
    plt.show()

In [ ]:
invert(all_image[3])
invert(all_image[4])
invert(all_image[5])

## Convolution

In [ ]:
def conv(img):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 20))
    kernel = np.ones((7, 7), np.float32)/25
    conv = cv2.filter2D(img, -1, kernel)
    ax[0].imshow(img)
    ax[0].set_title('Original Image', fontsize=24)
    ax[1].imshow(conv)
    ax[1].set_title('Convolved Image', fontsize=24)
    plt.show()
    

In [ ]:
conv(train_images[3])
conv(train_images[4])
conv(train_images[5])

## Blurring

In [ ]:
def blur(img):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 20))
    ax[0].imshow(img)
    ax[0].set_title('Original Image', fontsize=24)
    ax[1].imshow(cv2.blur(img, (100, 100)))
    ax[1].set_title('Blurred Image', fontsize=24)
    plt.show()

In [ ]:
blur(train_images[3])
blur(train_images[4])
blur(train_images[5])

## Modeling

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [ ]:
import visdom
vis=visdom.Visdom()
vis.close(env="main")

## define value tracker

In [ ]:
def value_tracker(value_plot, value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
             Y=value,
             win = value_plot,
             update='append'
             )

## Load train data

In [ ]:
def read_train_info():
    fp=train_info_pkl
    if False and os.path.exists(fp):
        try:
            print('loading train data from csv', flush=True)
            df_train=pd.read_csv(fp)
            print('complete!', flush=True)
        except EOFError:
            print('EOFError raised.', flush=True)
        
    else:
        df_train=pd.read_csv(data_path+"/train.csv")
        df_train["CAT"]=df_train[['healthy', 'multiple_diseases', 'rust','scab']].values.argmax(axis=1)
        df_train["id"]=df_train["image_id"].apply(lambda x : int(x.split("_")[1]))        
        X=df_train['id'].values
        y=df_train['CAT'].values
        skf=StratifiedKFold(n_splits=5,random_state=22)        
        df_train["fold"]=-1
        sum=0
        for i, (trn_idx,vld_idx) in enumerate(skf.split(X,y)):
            df_train.loc[vld_idx,"fold"]=i
        df_train.to_csv(train_info_pkl,index=False)
        
    return df_train

df_train=read_train_info()


## Augmentation

In [ ]:
import torchvision.transforms as transforms

transform_train = transforms.Compose([
#    transforms.RandomCrop(32, padding=4),
    transforms.Resize((img_rows,img_cols),interpolation=Image.NEAREST),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

transform_valid = transforms.Compose([
    transforms.Resize((img_rows,img_cols),interpolation=Image.NEAREST),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])
invTrans = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.5, 1/0.5, 1/0.5 ]),
                                transforms.Normalize(mean = [ -0.5, -0.5, -0.5 ],
                                                     std = [ 1., 1., 1. ]),
                               ])

In [ ]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose, Rotate, Cutout, HorizontalFlip, Normalize
)
from albumentations.pytorch import ToTensor, ToTensorV2
transform_train_al= Compose([
    Rotate(20),
    ToTensor()
])
transform_valid_al=Compose([
    ToTensor()
])

## Dataset

In [ ]:
from torch.utils.data import Dataset
class PlantDataset(Dataset):
    def __init__(self, df, tr=None):
        self.df = df
        self.tr=tr
        
    def __len__(self):
        return self.df.shape[0]
    def __getitem__(self, idx):
        name=self.df.iloc[idx].image_id
        image_src=img_path+"/"+name+".jpg"
        image = Image.open(image_src)
        labels = self.df.iloc[idx].CAT

        if self.tr is not None:
            image=self.tr(image)
        
        return image, labels

## transforms and normalization

In [ ]:
sel=1
trn_fold=[i for i in range(5) if i not in [sel]]
val_fold=[i for i in range(5) if i in [sel]]
trn_idx=df_train[df_train.fold.isin(trn_fold)].index
val_idx=df_train[df_train.fold.isin(val_fold)].index
print(trn_idx.shape,val_idx.shape)

In [ ]:
trainset = PlantDataset(df=df_train.loc[trn_idx],
                       tr=transform_train)
validset =PlantDataset(df=df_train.loc[val_idx],
                      tr=transform_valid)

train_loader = torch.utils.data.DataLoader(trainset,
                                          batch_size=batch_size,
                                          shuffle=True, num_workers=0)
valid_loader = torch.utils.data.DataLoader(validset,
                                          batch_size=batch_size,
                                          shuffle=False, num_workers=0)

In [ ]:
for img,lab in train_loader:
    break

In [ ]:
plt.imshow(img[0].permute(1,2,0))

## Model

In [ ]:
# import torchvision.models.resnet as resnet

In [ ]:
# conv1x1=resnet.conv1x1
# Bottleneck = resnet.Bottleneck
# BasicBlock= resnet.BasicBlock

In [ ]:
# class ResNet(nn.Module):

#     def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
#         super(ResNet, self).__init__()
#         self.inplanes = 16
#         self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1,
#                                bias=False)
#         self.bn1 = nn.BatchNorm2d(16)
#         self.relu = nn.ReLU(inplace=True)
#         #self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
#         self.layer1 = self._make_layer(block, 16, layers[0], stride=1)
#         self.layer2 = self._make_layer(block, 32, layers[1], stride=1)
#         self.layer3 = self._make_layer(block, 64, layers[2], stride=2)
#         self.layer4 = self._make_layer(block, 128, layers[3], stride=2)
        
#         self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
#         self.fc = nn.Linear(128 * block.expansion, num_classes)

#         for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
#             elif isinstance(m, nn.BatchNorm2d):
#                 nn.init.constant_(m.weight, 1)
#                 nn.init.constant_(m.bias, 0)

#         # Zero-initialize the last BN in each residual branch,
#         # so that the residual branch starts with zeros, and each residual block behaves like an identity.
#         # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
#         if zero_init_residual:
#             for m in self.modules():
#                 if isinstance(m, Bottleneck):
#                     nn.init.constant_(m.bn3.weight, 0)
#                 elif isinstance(m, BasicBlock):
#                     nn.init.constant_(m.bn2.weight, 0)

#     def _make_layer(self, block, planes, blocks, stride=1):
#         downsample = None
#         if stride != 1 or self.inplanes != planes * block.expansion:
#             downsample = nn.Sequential(
#                 conv1x1(self.inplanes, planes * block.expansion, stride),
#                 nn.BatchNorm2d(planes * block.expansion),
#             )

#         layers = []
#         layers.append(block(self.inplanes, planes, stride, downsample))
#         self.inplanes = planes * block.expansion
#         for _ in range(1, blocks):
#             layers.append(block(self.inplanes, planes))

#         return nn.Sequential(*layers)

#     def forward(self, x):
#         #[-1,3,32,32]
#         x = self.conv1(x)
#         #x.shape =[1, 16, 32,32]
#         x = self.bn1(x)
#         x = self.relu(x)
#         #x = self.maxpool(x)

#         x = self.layer1(x)
#         #x.shape =[1, 128, 32,32]
#         x = self.layer2(x)
#         #x.shape =[1, 256, 32,32]
#         x = self.layer3(x)
#         #x.shape =[1, 512, 16,16]
#         x = self.layer4(x)
#         #x.shape =[1, 1024, 8,8]
        
#         x = self.avgpool(x)
#         x = x.view(x.size(0), -1)
#         x = self.fc(x)

#         return x

In [ ]:
import torchvision.models  as models
import torch.nn as nn

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'
resnet34 = models.resnet34(pretrained=True)
num_ftrs=resnet34.fc.in_features
resnet34.fc=nn.Linear(num_ftrs,num_classes)

In [ ]:
resnet34=resnet34.to(device)

In [ ]:
a=torch.Tensor(1,3,img_rows,img_cols).to(device)
out = resnet34(a)
print(out)

In [ ]:
criterion=nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.SGD(resnet34.parameters(),lr=1e-2,momentum=0.9,weight_decay=5e-4)
lr_sched=optim.lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.5)

## make plot

In [ ]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),
                    opts=dict(title='loss_tracker', 
                              legend=['loss'], 
                              showlegend=True))
acc_plt = vis.line(Y=torch.Tensor(1).zero_(),
                   opts=dict(title='Accuracy', 
                             legend=['Acc'],
                             showlegend=True))

## define acc check func

In [ ]:
best_acc=0
def acc_check(net, test_set, epoch, save=1):
    correct = 0
    total = 0
    global best_acc
    with torch.no_grad():
        for data in test_set:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)

            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    acc = (100 * correct / total)
    print('Accuracy of the network on the 10000 test images: %d %%' % acc)
    if save and best_acc<acc:
        best_acc=acc
        torch.save(net.state_dict(), file_path+"_epoch_{}_acc_{}.pth".format(epoch, int(acc)))
    return acc

## train

In [ ]:

print(len(train_loader))
epochs = 10

for epoch in range(epochs):  # loop over the dataset multiple times
    
    running_loss = 0.0
    lr_sched.step()
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = resnet34(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 30 == 29:    # print every 30 mini-batches
            value_tracker(loss_plt, torch.Tensor([running_loss/30]), torch.Tensor([i + epoch*len(train_loader) ]))
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0
    
    #Check Accuracy
    acc = acc_check(resnet34, valid_loader, epoch, save=1)
    value_tracker(acc_plt, torch.Tensor([acc]), torch.Tensor([epoch]))
    

print('Finished Training')

In [ ]:
print(len(train_loader))

In [ ]:
torch.cuda.current_stream()

In [ ]:
torch.cuda.device_count()

In [ ]:
torch.cuda.empty_cache()